# Indice espectrales EVI en GEE

In [1]:
# Importar paquete GEE
import ee
ee.Initialize()

## Filtro zona estudio mediante GEE

In [2]:
# Agregar un shp GEE
zona_ee = ee.FeatureCollection("users/bravomoralesnino/SHP/ZONA_19S")

In [3]:
# Importar geemap
import geemap
Map = geemap.Map(basemap='SATELLITE')

In [4]:
geometria = zona_ee.geometry()
Map.centerObject(geometria, 11)

In [5]:
Map.addLayer(zona_ee, {"color" : "00FF11"},name = "Zona")
Map

Map(center=[-12.908744902436744, -69.9947155021359], controls=(WidgetControl(options=['position', 'transparent…

## Seleccionar Imagenes Landsat RS

In [6]:
# Seleccionar una imagen Landsat8
img_L8 = ee.Image("LANDSAT/LC08/C01/T1_SR/LC08_003069_20190808")\
                  .select(["B2","B3","B4","B5","B6","B7"],
                          ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2'])\
                  .multiply(0.0001)\
                  .reproject(crs = "EPSG:32719", scale = 30)

In [7]:
# Seleccionar una imagen Sentinel2
img_S2 = ee.Image("COPERNICUS/S2_SR/20190808T145729_20190808T145732_T19LCF")\
                  .select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12'],
                          ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2'])\
                  .multiply(0.0001) # 0 - 1

## Consultar nombre bandas y proyección

In [9]:
# Nombre de las bandas
print("Nombre bandas L8: ",img_L8.bandNames().getInfo())
print("Nombre bandas S2: ",img_S2.bandNames().getInfo())

Nombre bandas L8:  ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']
Nombre bandas S2:  ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']


In [10]:
# Consultar Proyeccion
print("Proyeccion L8: ",img_L8.projection().getInfo())
print("Proyeccion S2: ",img_S2.select(['BLUE']).projection().getInfo())

Proyeccion L8:  {'type': 'Projection', 'crs': 'EPSG:32719', 'transform': [30, 0, 0, 0, -30, 0]}
Proyeccion S2:  {'type': 'Projection', 'crs': 'EPSG:32719', 'transform': [10, 0, 300000, 0, -10, 8600020]}


In [11]:
# Simbologia estala 0 - 1
viz_es = {
    'bands': ['SWIR1','NIR','RED'],
    'min': 0.1,
    'max': 0.6,
    'gamma': 1.6
}

## Recortar imagen zona

In [12]:
# Recortar informacion con la zona de estudio
img_L8_clip = img_L8.clip(zona_ee)
img_S2_clip = img_S2.clip(zona_ee)

In [13]:
# Visualizar 
Map.addLayer(img_L8_clip, viz_es, name = "Landsat8 RS")
Map.addLayer(img_S2_clip, viz_es, name = "Sentinel2 RS")
Map

Map(bottom=281405.0, center=[-12.908572686061014, -70.09410751022057], controls=(WidgetControl(options=['posit…

## Determinar EVI

In [14]:
# Determinar EVI Funcion GEE
def Calc_EVI(image):
    evi = image.expression("(2.5 * float(nir - red)/(nir + 6*red - 7.5*blue + 1))", {
        'blue': image.select('BLUE'),
        'nir': image.select('NIR'),
        'red': image.select('RED')
    })
    return evi.rename("EVI")

In [15]:
# Calcular indices EVI con funcion
evi_L8 = Calc_EVI(img_L8_clip)
evi_S2 = Calc_EVI(img_S2_clip)

In [16]:
# Simbologia de EVI
viz_evi = {
    'palette': ['FFFFFF', 'CE7E45','DF923D', 'F1B555',
                'FCD163', '99B718','74A901', '66A000',
                '529400', '3E8601','207401', '056201',
                '004C00', '023B01','012E01', '011D01',
                '011301'],
    'min': -0.3,
    'max': 0.8,
    'bands': "EVI"
}

In [17]:
# Visualizar Mapa
Map.addLayer(evi_L8, viz_evi, name = "Landsat8 EVI")
Map.addLayer(evi_S2, viz_evi, name = "Sentinel2 EVI")
Map

Map(bottom=4497480.0, center=[-12.887951161916048, -70.02155542373659], controls=(WidgetControl(options=['posi…

## Clasificacion de EVI 
- EVI < -0.1  = 1
- EVI >= 0.1  = 2
- EVI >= 0.45 = 3

In [18]:
L8_clas_evi = ee.Image(0).where(evi_L8.lt(-0.1),1)\
                      .where(evi_L8.gte(-0.1),2)\
                      .where(evi_L8.gte(0.45),3)\
                      .clip(zona_ee)

In [19]:
# Simbologia de clasificacion de EVI 3 clases
viz_clas = {
    'palette': ["#ff03e0",'#fffa53',"#23941c"],
    'min': 1,
    'max': 3
}

In [20]:
# Visualizar 
Map.addLayer(L8_clas_evi , viz_clas, name = "L8 EVI Class")
Map.add_colorbar(viz_clas, label="Clase EVI", layer_name="EVI")
Map

Map(bottom=281456.0, center=[-12.942998943409737, -69.91184234619142], controls=(WidgetControl(options=['posit…

## Exportar Imagen

In [21]:
import os
os.getcwd()

'C:\\Users\\GEOMATICA\\Teledeteccion_Python\\GEE'

In [22]:
ruta_archivos = r"E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\Resultado_GEE'

In [23]:
# Exportar imagen directorio
geemap.ee_export_image(L8_clas_evi, filename="L8_clas_evi.tif", scale=30, region=geometria, file_per_band=False)
geemap.ee_export_image(evi_L8, filename="L8_evi.tif", scale=30, region=geometria, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE\L8_clas_evi.tif
Generating URL ...
Please wait ...
Data downloaded to E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE\L8_evi.tif
